# Scraping Hotel Ratings on Tripadvisor

In this homework we will practice web scraping. Let's get some basic information for each hotel in Boston.

On each hotel page, scrape the Traverler ratings. **(10 pts)**

![Information to be scraped](traveler_ratings.png)

Save the data in "traverler_ratings.csv" in the following format:

hotel_name, rating, count

In [2]:
from bs4 import BeautifulSoup
import sys
import time
import os
import logging
import argparse
import requests
import codecs
import json
import pandas as pd

base_url = "http://www.tripadvisor.com"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.76 Safari/537.36"
df_names = pd.DataFrame(columns=['name'])
df_names_final = pd.DataFrame(columns=['name'])
df_rnames = pd.DataFrame(columns=['rname'])
df_rnames_final = pd.DataFrame(columns=['rname'])
df_ratings = pd.DataFrame(columns=['rating'])
df_ratings_final = pd.DataFrame(columns=['rating'])

def get_tourism_page(city):
    """
        Return json containing the URL
        of the tourism city page
    """

    # EXAMPLE: http://www.tripadvisor.com/Boston
    url = base_url+ "/"+ city 
    
    # Given the url, request the HTML page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)
    
    html = response.text.encode('utf-8')

    soup = BeautifulSoup(html, "lxml")

    li = soup.find("link", {"hreflang": "en"})
    #return li['href']
    
    return 'https://www.tripadvisor.com/Tourism-g60745-Boston_Massachusetts-Vacations.html'
    
def get_city_page(tourism_url):
    """
        Get the URL of the hotels of the city
        using the URL returned by the function
        get_tourism_page()
        """

    url = tourism_url
    # Given the url, request the HTML page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)

    html = response.text.encode('utf-8')

    # Use BeautifulSoup to extract the url for the list of hotels in
    # the city and state we are interested in.
    # For exampel in this case we need to
    #<li class="hotels twoLines">
    #<a href="/Hotels-g60745-Boston_Massachusetts-Hotels.html" data-trk="hotels_nav"
    soup = BeautifulSoup(html,'lxml')

    li = soup.find("li", {"class": "hotels twoLines"})
    city_url = li.find('a', href = True)

    return city_url['href']

def get_hotellist_page(city_url):
    """ Get the hotel list page given the url returned by
        get_city_page(). Return the html after saving
        it to the datadir
    """

    url = base_url + city_url
    # Sleep 2 sec before starting a new http request
    time.sleep(2)
    # Request page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)
    html = response.text.encode('utf-8')
    return html

def hotel_traveller_rating(url):
    html = get_hotellist_page(url)
    soup = BeautifulSoup(html,'lxml')
    hotel_rating = soup.find('div',{'class':'col rating '})
    row_name = soup.select('div.row_label')
    
    rname = []
    for i in row_name:
        n = i.find(text=True)
        rname.append(n) 
    row_rating = hotel_rating.select('li')
    
    rating=[]
    names = []
    for i in row_rating:
        name = soup.find('div',{'class':'warLocName'})
        name= name.find(text=True)
        names.append(name)
        count = i.find('span',{'class':''}).find(text=True)
        rating.append(count)
        #print(j.find(text=True))
    df_names['name']=names
    df_rnames['rname']=rname
    df_ratings['rating']=rating
    global df_names_final
    global df_rnames_final
    global df_ratings_final
    df_names_final = df_names_final.append(df_names,ignore_index=True)
    df_rnames_final = df_rnames_final.append(df_rnames,ignore_index=True)
    df_ratings_final = df_ratings_final.append(df_ratings,ignore_index=True)

def parse_hotellist_page(html):
    """ Parse the html pages returned by get_hotellist_page().
        Return the next url page to scrape (a city can have
        more than one page of hotels) if there is, else exit
        the script.
    """
    soup = BeautifulSoup(html,'lxml')

    # Extract hotel name, star rating and number of reviews
    hotel_boxes = soup.select('div.listing.easyClear.p13n_imperfect')
    
    # hotel_boxes = soup.find_all('div', {'class' :'listing easyClear  p13n_imperfect '})
    for hotel_box in hotel_boxes:
#         name = hotel_box.find('div', {'class' :'listing_title'}).find(text=True)
#         print("HOTEL NAME: {}".format(name))
        hotel_link = hotel_box.find('a', {'class' : 'property_title'})
        hotel_traveller_rating(hotel_link['href'])
        
    # Get next URL page if exists, else exit
    div = soup.find("div", {"class" : "unified pagination standard_pagination"})
    # check if last page
    if div.find('span', {'class' : 'nav next ui_button disabled'}):
        print("We reached last page") 
    # If it is not las page there must be the Next URL
    hrefs = div.findAll('a', href= True)
    for href in hrefs:
        if href.find(text = True) == 'Next':
            print("Next url is :",href['href'])
            return href['href']


In [3]:
# Obtain the url of the toursim page 
tourism_url = get_tourism_page('Boston')
    
#Get URL to obtaint the list of hotels in a specific city
city_url = get_city_page(tourism_url)

c=0
while(True):
    c +=1
    if city_url == None:
        break
    html = get_hotellist_page(city_url)
    city_url = parse_hotellist_page(html)


Next url is : /Hotels-g60745-oa30-Boston_Massachusetts-Hotels.html#ACCOM_OVERVIEW
Next url is : /Hotels-g60745-oa60-Boston_Massachusetts-Hotels.html#ACCOM_OVERVIEW
We reached last page


In [1]:
df_final = pd.DataFrame()
df_final['hotel_name'] = df_names_final['name']
df_final['rating'] = df_rnames_final['rname'] 
df_final['count'] = df_ratings_final['rating']
print(df_final)
df_final.to_csv('traveller_ratings.csv',index=False)
print("done")

NameError: name 'pd' is not defined

-------

Next, scrape all the reviews of each hotel for the star ratings of the following attributes: Value, Location, Sleep Quality, Rooms, Cleanliness, Service. Note that some reviews may not have attribute ratings and some may only have some of the attributes. **(25 pts)**

![Information to be scraped](attribute_ratings.png)

Save the data in "attribute_ratings.csv" in the following format:

hotel_name, review_id, attribute, star_value

In [21]:
from bs4 import BeautifulSoup
import sys
import time
import os
import logging
import argparse
import requests
import codecs
import json
import pandas as pd

base_url = "http://www.tripadvisor.com"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.76 Safari/537.36"

hname = []
rid = []
attr = []
svalue = []

def get_tourism_page(city):
    """
        Return json containing the URL
        of the tourism city page
    """

    # EXAMPLE: http://www.tripadvisor.com/Boston
    url = base_url+ "/"+ city 
    
    # Given the url, request the HTML page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)
    
    html = response.text.encode('utf-8')

    soup = BeautifulSoup(html, "lxml")

    li = soup.find("link", {"hreflang": "en"})
    
    #print(li)
    #return li['href']
    return 'https://www.tripadvisor.com/Tourism-g60745-Boston_Massachusetts-Vacations.html'

def get_city_page(tourism_url):
    """
        Get the URL of the hotels of the city
        using the URL returned by the function
        get_tourism_page()
        """

    url = tourism_url
    # Given the url, request the HTML page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)

    html = response.text.encode('utf-8')

    # Use BeautifulSoup to extract the url for the list of hotels in
    # the city and state we are interested in.
    # For exampel in this case we need to
    #<li class="hotels twoLines">
    #<a href="/Hotels-g60745-Boston_Massachusetts-Hotels.html" data-trk="hotels_nav"
    soup = BeautifulSoup(html,'lxml')

    li = soup.find("li", {"class": "hotels twoLines"})
    city_url = li.find('a', href = True)

    return city_url['href']

def get_hotellist_page(city_url):
    """ Get the hotel list page given the url returned by
        get_city_page(). Return the html after saving
        it to the datadir
    """

    url = base_url + city_url
    # Sleep 2 sec before starting a new http request
    time.sleep(2)
    # Request page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)
    html = response.text.encode('utf-8')
    return html

def extract_reviewr(url, hotel_name,sleep):
    html = get_hotellist_page(url)
    soup = BeautifulSoup(html,'lxml')
    
    review_id = soup.select('div.reviewSelector')
    for m in review_id:
        review_rating = m.select('div.rating-list')
        for s in review_rating:
            col_1 = s.find('li')
            if col_1 == None:
                continue
            rating = col_1.select('img')
            rating_name = col_1.select('div.recommend-description')
            for i,j in zip(rating_name,rating):
                hname.append(hotel_name)
                rid.append(m['id'])
                attr.append(i.find(text=True))
                svalue.append(j['alt'][0])
    div = soup.find('div',{'class':'unified pagination '})
    if div.find('span',{'class':'nav next disabled'}):
        print('Reached last page')
    else:
        sleep+=1
        if sleep%50 == 0:
            time.sleep(1)
        button = div.find('a',{'class':'nav next rndBtn ui_button primary taLnk'})
        extract_reviewr(button['href'], hotel_name,sleep)
    
def hotel_traveller_review(url):
    html = get_hotellist_page(url)
    soup = BeautifulSoup(html,'lxml')
    hotel_name = soup.find('h1').find(text=True)
    hotel_reviews = soup.find('div',{'class':'innerBubble'})
    review_link = hotel_reviews.find('a')
    extract_reviewr(review_link['href'], hotel_name,0)

    
def parse_hotellist_page(html):
    """ Parse the html pages returned by get_hotellist_page().
        Return the next url page to scrape (a city can have
        more than one page of hotels) if there is, else exit
        the script.
    """
    soup = BeautifulSoup(html,'lxml')

    # Extract hotel name, star rating and number of reviews
    hotel_boxes = soup.select('div.listing.easyClear.p13n_imperfect')
    
    # hotel_boxes = soup.find_all('div', {'class' :'listing easyClear  p13n_imperfect '})
    for hotel_box in hotel_boxes:
        hotel_link = hotel_box.find('a', {'class' : 'property_title'})
        hotel_traveller_review(hotel_link['href'])
        
    # Get next URL page if exists, else exit
    div = soup.find("div", {"class" : "unified pagination standard_pagination"})
    # check if last page
    if div.find('span', {'class' : 'nav next ui_button disabled'}):
        print("We reached last page") 
    # If it is not las page there must be the Next URL
    hrefs = div.findAll('a', href= True)
    for href in hrefs:
        if href.find(text = True) == 'Next':
            print("Next url is :",href['href'])
            return href['href']


In [ ]:
# Obtain the url of the toursim page 
city = 'Boston'
tourism_url = get_tourism_page(city)
    
#Get URL to obtaint the list of hotels in a specific city
city_url = get_city_page(tourism_url)

c=0
while(True):
    
    c +=1
    if city_url == None:
        break
    html = get_hotellist_page(city_url)
    city_url = parse_hotellist_page(html)


In [3]:
final_df = pd.DataFrame()
final_df['hotel_name'] = hname
final_df['review_id'] = rid
final_df['attribute'] = attr
final_df['star_value'] = svalue
print(len(final_df))
final_df.to_csv('attribute_ratings.csv',index=False)

220717


In [11]:
"Continuing from the last hotel when jupyter notebook crashed"

from bs4 import BeautifulSoup
import sys
import time
import os
import logging
import argparse
import requests
import codecs
import json
import pandas as pd
import csv

base_url = "http://www.tripadvisor.com"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.76 Safari/537.36"
jump = 0

def get_tourism_page():
    return 'https://www.tripadvisor.com/Tourism-g60745-Boston_Massachusetts-Vacations.html'

def get_city_page(tourism_url):
    """
        Get the URL of the hotels of the city
        using the URL returned by the function
        get_tourism_page()
        """

    url = tourism_url
    # Given the url, request the HTML page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)

    html = response.text.encode('utf-8')

    # Use BeautifulSoup to extract the url for the list of hotels in
    # the city and state we are interested in.
    # For exampel in this case we need to
    #<li class="hotels twoLines">
    #<a href="/Hotels-g60745-Boston_Massachusetts-Hotels.html" data-trk="hotels_nav"
    soup = BeautifulSoup(html,'lxml')

    li = soup.find("li", {"class": "hotels twoLines"})
    city_url = li.find('a', href = True)

    return city_url['href']

def get_hotellist_page(city_url):
    """ Get the hotel list page given the url returned by
        get_city_page(). Return the html after saving
        it to the datadir
    """

    url = base_url + city_url
    # Sleep 2 sec before starting a new http request
    time.sleep(2)
    # Request page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)
    html = response.text.encode('utf-8')
    return html


def hotel_traveller_review(url):
    html = get_hotellist_page(url)
    soup = BeautifulSoup(html,'lxml')
    hotel_name = soup.find('h1').find(text=True)
    hotel_reviews = soup.find('div',{'class':'innerBubble'})
    review_link = hotel_reviews.find('a')
    extract_reviewr(review_link['href'], hotel_name,0)

def extract_reviewr(url, hotel_name,sleep):
    html = get_hotellist_page(url)
    soup = BeautifulSoup(html,'lxml')
    
    review_id = soup.select('div.reviewSelector')
    for m in review_id:
        review_rating = m.select('div.rating-list')
        for s in review_rating:
            col_1 = s.find('li')
            if col_1 == None:
                continue
            rating = col_1.select('img')
            rating_name = col_1.select('div.recommend-description')
            for i,j in zip(rating_name,rating):
                with open('attribute_ratings.csv', 'a', newline='') as file:
                    writer = csv.writer(file,quoting=csv.QUOTE_MINIMAL)
                    writer.writerow([hotel_name,m['id'],i.find(text=True),j['alt'][0]])
    div = soup.find('div',{'class':'unified pagination '})
    if div.find('span',{'class':'nav next disabled'}):
        print('Reached last page')
    else:
        sleep+=1
        if sleep%50 == 0:
            time.sleep(1)
        button = div.find('a',{'class':'nav next rndBtn ui_button primary taLnk'})
        extract_reviewr(button['href'], hotel_name,sleep)


def parse_hotellist_page_from_current(html,jump):
    """ Parse the html pages returned by get_hotellist_page().
        Return the next url page to scrape (a city can have
        more than one page of hotels) if there is, else exit
        the script.
    """
    soup = BeautifulSoup(html,'lxml')

    # Extract hotel name, star rating and number of reviews
    hotel_boxes = soup.select('div.listing.easyClear.p13n_imperfect')
    
    # hotel_boxes = soup.find_all('div', {'class' :'listing easyClear  p13n_imperfect '})
    for hotel_box in hotel_boxes:
        current_name = hotel_box.find('div', {'class' :'listing_title'}).find(text=True)
        print(current_name)
        if jump == 0:
            if current_name != last_hotel:
                continue
            else:
                jump = 1
        hotel_link = hotel_box.find('a', {'class' : 'property_title'})
        hotel_traveller_review(hotel_link['href'])
            
    # Get next URL page if exists, else exit
    div = soup.find("div", {"class" : "unified pagination standard_pagination"})
    # check if last page
    if div.find('span', {'class' : 'nav next ui_button disabled'}):
        print("We reached last page") 
    # If it is not las page there must be the Next URL
    hrefs = div.findAll('a', href= True)
    for href in hrefs:
        if href.find(text = True) == 'Next':
            print("Next url is :",href['href'])
            return href['href']


last_hotel = "Element Boston Seaport"
tourism_url = get_tourism_page()
city_url = get_city_page(tourism_url)

while(city_url!=None):
    html = get_hotellist_page(city_url)
    city_url = parse_hotellist_page_from_current(html,jump)

Marriott Vacation Club Pulse at Custom House, Boston
Boston Harbor Hotel
Seaport Boston Hotel
Four Seasons Hotel Boston
Lenox Hotel
Courtyard Boston Copley Square
InterContinental Boston
Hotel Commonwealth
Mandarin Oriental, Boston
Residence Inn Boston Back Bay/Fenway
Eliot Hotel
XV Beacon
Residence Inn Boston Downtown/Seaport
Kimpton Nine Zero Hotel
Hilton Garden Inn Boston Logan Airport
The Verb Hotel
The Godfrey Hotel Boston
enVision Hotel Boston - Longwood
The Langham, Boston
Loews Boston Hotel
The Boxer Boston
Colonnade Hotel
Residence Inn Boston Harbor on Tudor Wharf
Fairmont Copley Plaza, Boston
Copley Square Hotel
W Boston
The Bostonian Boston
Taj Boston
The Westin Copley Place, Boston
Newbury Guest House
Next url is : /Hotels-g60745-oa30-Boston_Massachusetts-Hotels.html#ACCOM_OVERVIEW
The Ritz-Carlton, Boston
The Envoy Hotel, Autograph Collection
Hyatt Regency Boston Harbor
Hyatt Regency Boston
Kimpton Onyx Hotel
Boston Marriott Long Wharf
The Liberty, A Luxury Collection Hote

-------